### MAPPATURA POSIZIONI ALTERNATIVE E PLAYSYLE PLUS

In [ ]:
import pandas as pd
import numpy as np

# Carichiamo il dataset
try:
    df_movimento = pd.read_csv('../data/puliti/movimento_puliti.csv')
except FileNotFoundError:
    print("Il file 'movimento_puliti.csv' non è stato trovato.")
    exit(1)

# --- 1 MAPPATURE PER POSIZIONI ---
# Definiamo le 3 macro-aree
POS_ATT = {'ST', 'CF', 'LW', 'RW', 'RF', 'LF'}
POS_CEN = {'CM', 'CDM', 'CAM', 'LM', 'RM'}
POS_DIF = {'CB', 'LB', 'RB', 'LWB', 'RWB'}

# --- 2 MAPPATURE PER PLAYSTYLES ---
# Definiamo le mappature per i playstyles (escludiamo i portieri)
PLAYSTYLE_CATEGORIES = {
    'ps_scoring': ['Finesse Shot', 'Power Header', 'Power Shot', 'Chip Shot', 'Dead Ball'],
    'ps_passing': ['Pinged Pass', 'Incisive Pass', 'Long Ball Pass', 'Tiki Taka', 'Whipped Cross'],
    'ps_control': ['First Touch', 'Press Proven', 'Trickster', 'Flair', 'Technical'],
    'ps_defend': ['Block', 'Bruiser', 'Intercept', 'Jockey', 'Slide Tackle', 'Anticipate'],
    'ps_physical': ['Acrobatic', 'Aerial', 'Trivela', 'Relentless', 'Quick Step']
}

### CREAZIONE FEATURE POSIZIONI ALTERNATIVE

In [ ]:
def processa_posizoni(alternate_pos_str):
    """
    Funzione helper per contare le posizioni alternative
    basate sulle macro-aree definite.
    """
    # valor di default
    counts = {'att': 0, 'cen': 0, 'dif': 0}

    # Controlla se la stringa è NaN o vuota
    if pd.isna(alternate_pos_str) or alternate_pos_str.strip() == '':
        return counts ['att'], counts['cen'], counts['dif']
    
    # Pulisce la lista e Split della stringa in una lista di posizioni
    positions = [pos.strip() for pos in alternate_pos_str.split(',')]

    for pos in positions:
        if pos in POS_ATT:
            counts['att'] += 1
        elif pos in POS_CEN:
            counts['cen'] += 1
        elif pos in POS_DIF:
            counts['dif'] += 1

    return counts['att'], counts['cen'], counts['dif']

print("Inizio elaborazione delle posizioni alternative...")
# Applichiamo la funzione al DataFrame
df_movimento[['num_alt_pos_att', 'num_alt_pos_cen', 'num_alt_pos_dif']] = df_movimento['Alternative positions'].apply(
    lambda x: pd.Series(processa_posizoni(x))
)
print("Elaborazione delle posizioni alternative completata.")


Inizio elaborazione delle posizioni alternative...
Elaborazione delle posizioni alternative completata.


### CREAZIONE FEATURE PLAYSTYLE

In [6]:
def processa_playstyles(playstyles_str):
    """
    Funzione helper per contare i playstyles regolari e plus
    basati su categorie definite.
    """
    # inizializza i contatori
    counts = {
        'ps_scoring': 0, 'ps_scoring_plus': 0,
        'ps_passing': 0, 'ps_passing_plus': 0,
        'ps_control': 0, 'ps_control_plus': 0,
        'ps_defend': 0, 'ps_defend_plus': 0,
        'ps_physical': 0, 'ps_physical_plus': 0
    }

    # Controlla se il giocatore ha playstyles
    if pd.isna(playstyles_str) or playstyles_str.strip() == '':
        return pd.Series(counts)
    
    # Pulisce la lista e Split della stringa in una lista di playstyles
    playstyles = [ps.strip() for ps in playstyles_str.split(',')]

    for ps_full in playstyles:
        is_plus = False

        # Controlla se è un playstyle plus
        if ps_full.endswith('+'):
            is_plus = True
            ps_name = ps_full[:-1]

        else:
            ps_name = ps_full

        # Determina la categoria del playstyle
        for category_key, ps_in_category_list in PLAYSTYLE_CATEGORIES.items():
            if ps_name in ps_in_category_list:

                if is_plus:
                    counts[f'{category_key}_plus'] += 1
                else:
                    counts[category_key] += 1
                break

    return pd.Series(counts)

print("Inizio elaborazione dei playstyles...")
# Applichiamo la funzione al DataFrame
df_movimento_playstyles = df_movimento['play style'].apply(processa_playstyles)

# Uniamo i nuovi dati al DataFrame originale
df_movimento = pd.concat([df_movimento, df_movimento_playstyles], axis=1)
print("Elaborazione dei playstyles completata.")

Inizio elaborazione dei playstyles...
Elaborazione dei playstyles completata.


### VERIFICA FINALE

In [7]:
colonne_posizioni = ['num_alt_pos_att', 'num_alt_pos_cen', 'num_alt_pos_dif']
colonne_playstyles = list(df_movimento_playstyles.columns)

print("Verifica finale delle nuove feature...")
print(df_movimento[df_movimento['Alternative positions'].notna()][['Alternative positions'] + colonne_posizioni].head())

print("Verifica finale dei playstyles...")
print(df_movimento[df_movimento['play style'].notna()][['play style'] + colonne_playstyles].head())

print("Verifica tipi di dato...")
print(df_movimento[colonne_posizioni + colonne_playstyles].dtypes)

Verifica finale delle nuove feature...
  Alternative positions  num_alt_pos_att  num_alt_pos_cen  num_alt_pos_dif
0                    LW                1                0                0
1                    CM                0                1                0
3                    CM                0                1                0
4                ST, LM                1                1                0
5                   CAM                0                1                0
Verifica finale dei playstyles...
                                          play style  ps_scoring  \
0  Quick Step+, Acrobatic, Finesse Shot, Flair, R...           1   
1  Tiki Taka+, Aerial, Bruiser, Long Ball Pass, P...           1   
2  Acrobatic+, Bruiser, Power Header, Power Shot,...           2   
3  Relentless+, Flair, Intercept, Slide Tackle, T...           0   
4  Quick Step+, Chip Shot, Finesse Shot, First To...           2   

   ps_scoring_plus  ps_passing  ps_passing_plus  ps_control  ps_cont

### SALVATAGGIO 

In [ ]:
# --- SALVATAGGIO NUOVO DF ---

output_filename = '../data/puliti/movimento_fe.csv'
df_movimento.to_csv(output_filename, index=False)

print(f"DataFrame arricchito salvato con successo in: {output_filename}")
print(f"Contiene {df_movimento.shape[0]} righe e {df_movimento.shape[1]} colonne.")

DataFrame arricchito salvato con successo in: movimento_fe.csv
Contiene 15738 righe e 62 colonne.
